In [ ]:
from __future__ import print_function
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2
import skimage.morphology as morp
from skimage.filters import rank
from sklearn.utils import shuffle
import csv
from PIL import Image
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torchvision import datasets, transforms
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

!mkdir data
%cd data

/content/data


In [ ]:
!wget https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
!unzip traffic-signs-data.zip
# ! wget https://raw.githubusercontent.com/georgesung/traffic_sign_classification_german/master/signnames.csv

--2021-03-04 02:59:21--  https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 99.86.227.60, 99.86.227.118, 99.86.227.221, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|99.86.227.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123524425 (118M) [application/zip]
Saving to: ‘traffic-signs-data.zip’

traffic-signs-data. 100%[===================>] 117.80M  17.4MB/s    in 6.0s    

2021-03-04 02:59:27 (19.8 MB/s) - ‘traffic-signs-data.zip’ saved [123524425/123524425]

Archive:  traffic-signs-data.zip
  inflating: test.p                  
  inflating: train.p                 
  inflating: valid.p                 


In [ ]:
# data augmentation for training and test time
# Resize all images to 32 * 32 and normalize them to mean = 0 and standard-deviation = 1 based on statistics collected from the training set

data_transforms = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and jitter image brightness
data_jitter_brightness = transforms.Compose([
	  transforms.Resize((32, 32)),
    transforms.ColorJitter(brightness=5),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and jitter image saturation
data_jitter_saturation = transforms.Compose([
	  transforms.Resize((32, 32)),
    transforms.ColorJitter(saturation=5),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and jitter image contrast
data_jitter_contrast = transforms.Compose([
	  transforms.Resize((32, 32)),
    transforms.ColorJitter(contrast=5),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and jitter image hues
data_jitter_hue = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.ColorJitter(hue=0.4),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and rotate image
data_rotate = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and flip image horizontally and vertically
data_hvflip = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(1),
    transforms.RandomVerticalFlip(1),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and flip image horizontally
data_hflip = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(1),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and flip image vertically
data_vflip = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.RandomVerticalFlip(1),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and shear image
data_shear = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.RandomAffine(degrees = 15,shear=2),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and translate image
data_translate = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.RandomAffine(degrees = 15,translate=(0.1,0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and crop image 
data_center = transforms.Compose([
	transforms.Resize((36, 36)),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

# Resize, normalize and convert image to grayscale
data_grayscale = transforms.Compose([
	transforms.Resize((32, 32)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

In [ ]:
training_file = "./train.p"
validation_file= "./valid.p"
testing_file = "./test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

# signs = []
# with open('signnames.csv', 'r') as csvfile:
#     signnames = csv.reader(csvfile, delimiter=',')
#     next(signnames,None)
#     for row in signnames:
#         signs.append(row[1])
#     csvfile.close()

X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

# Number of training examples
n_train = X_train.shape[0]

# Number of testing examples
n_test = X_test.shape[0]

# Number of validation examples.
n_validation = X_valid.shape[0]

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples: ", n_train)
print("Number of testing examples: ", n_test)
print("Number of validation examples: ", n_validation)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples:  34799
Number of testing examples:  12630
Number of validation examples:  4410
Image data shape = (32, 32, 3)
Number of classes = 43


In [ ]:
for num, image in enumerate(X_train):
    image = np.ascontiguousarray(image)
    img = Image.fromarray(image, 'RGB')
    if not os.path.exists(str(y_train[num])):
      os.system('mkdir -p train_images/{label}'.format(label = str(y_train[num])))
    img.save("./train_images/{label}/{i}.jpg".format(i = num, label = str(y_train[num])))
      

In [ ]:
for num, image in enumerate(X_test):
    image = np.ascontiguousarray(image)
    img = Image.fromarray(image, 'RGB')
    if not os.path.exists(str(y_train[num])):
      os.system('mkdir -p test_images/{label}'.format(label = str(y_test[num])))
    img.save("./test_images/{label}/{i}.jpg".format(i = num, label = str(y_test[num])))

In [ ]:
for num, image in enumerate(X_valid):
    image = np.ascontiguousarray(image)
    img = Image.fromarray(image, 'RGB')
    if not os.path.exists(str(y_train[num])):
      os.system('mkdir -p val_images/{label}'.format(label = str(y_valid[num])))
    img.save("./val_images/{label}/{i}.jpg".format(i = num, label = str(y_valid[num])))

In [ ]:
if torch.cuda.is_available():
    use_gpu = True
    print("Using GPU")
else:
	use_gpu = False
	print("Using CPU")
 
FloatTensor = torch.cuda.FloatTensor if use_gpu else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_gpu else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_gpu else torch.ByteTensor
Tensor = FloatTensor

Using GPU


In [ ]:
train_loader = torch.utils.data.DataLoader(
   torch.utils.data.ConcatDataset([datasets.ImageFolder('./train_images', 
   transform=data_transforms), datasets.ImageFolder('./train_images',
   transform=data_jitter_brightness),datasets.ImageFolder('./train_images',
   transform=data_jitter_hue),datasets.ImageFolder('./train_images',
   transform=data_jitter_contrast),datasets.ImageFolder('./train_images',
   transform=data_jitter_saturation),datasets.ImageFolder('./train_images',
   transform=data_translate),datasets.ImageFolder('./train_images',
   transform=data_rotate),datasets.ImageFolder('./train_images',
   transform=data_hvflip),datasets.ImageFolder('./train_images',
   transform=data_center),datasets.ImageFolder('./train_images',
   transform=data_shear)]), batch_size=64, shuffle=True, num_workers=4, pin_memory=use_gpu)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('./val_images',
                         transform=data_transforms),
    batch_size=64, shuffle=False, num_workers=4, pin_memory=use_gpu)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('./test_images',
                         transform=data_transforms),
    batch_size=64, shuffle=False, num_workers=4, pin_memory=use_gpu)

In [ ]:
nclasses = 43 # GTSRB as 43 classes

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # CNN layers
        self.conv1 = nn.Conv2d(3, 100, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(100)
        self.conv2 = nn.Conv2d(100, 150, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(150)
        self.conv3 = nn.Conv2d(150, 250, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(250)
        self.conv_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(250*2*2, 350)
        self.fc2 = nn.Linear(350, nclasses)

        self.localization = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=7),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True),
            nn.Conv2d(8, 10, kernel_size=5),
            nn.MaxPool2d(2, stride=2),
            nn.ReLU(True)
            )

        # Regressor for the 3 * 2 affine matrix
        self.fc_loc = nn.Sequential(
            nn.Linear(10 * 4 * 4, 32),
            nn.ReLU(True),
            nn.Linear(32, 3 * 2)
            )
   
        # Initialize the weights/bias with identity transformation
        self.fc_loc[2].weight.data.zero_()
        self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))


    # Spatial transformer network forward function
    def stn(self, x):
        xs = self.localization(x)
        xs = xs.view(-1, 10 * 4 * 4)
        theta = self.fc_loc(xs)
        theta = theta.view(-1, 2, 3)
        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def forward(self, x):
        # transform the input
        x = self.stn(x)

        # Perform forward pass
        x = self.bn1(F.max_pool2d(F.leaky_relu(self.conv1(x)),2))
        x = self.conv_drop(x)
        x = self.bn2(F.max_pool2d(F.leaky_relu(self.conv2(x)),2))
        x = self.conv_drop(x)
        x = self.bn3(F.max_pool2d(F.leaky_relu(self.conv3(x)),2))
        x = self.conv_drop(x)
        x = x.view(-1, 250*2*2)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [ ]:
model = Net()

if use_gpu:
    model.cuda()

In [ ]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad,model.parameters()),lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=5,factor=0.5,verbose=True)

In [ ]:
def train(epoch):
    model.train()
    correct = 0
    training_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        if use_gpu:
            data = data.cuda()
            target = target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        max_index = output.max(dim = 1)[1]
        correct += (max_index == target).sum()
        training_loss += loss
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss per example: {:.6f}\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), loss.data.item()/(640),loss.data.item()))
    print('\nTraining set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                training_loss / len(train_loader.dataset), correct, len(train_loader.dataset),
                100. * correct / len(train_loader.dataset)))


In [ ]:
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
            if use_gpu:
                data = data.cuda()
                target = target.cuda()
            output = model(data)
            validation_loss += F.nll_loss(output, target, size_average=False).data.item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    scheduler.step(np.around(validation_loss,2))
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

In [ ]:
for epoch in range(1, 11):
    train(epoch)
    validation()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3448: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


Train Epoch: 1 [0/347990 (0%)]	Loss per example: 0.005984	Loss: 3.830052
Train Epoch: 1 [640/347990 (0%)]	Loss per example: 0.006111	Loss: 3.911069
Train Epoch: 1 [1280/347990 (0%)]	Loss per example: 0.005893	Loss: 3.771404
Train Epoch: 1 [1920/347990 (1%)]	Loss per example: 0.005867	Loss: 3.754845
Train Epoch: 1 [2560/347990 (1%)]	Loss per example: 0.005699	Loss: 3.647320
Train Epoch: 1 [3200/347990 (1%)]	Loss per example: 0.005632	Loss: 3.604322
Train Epoch: 1 [3840/347990 (1%)]	Loss per example: 0.005876	Loss: 3.760393
Train Epoch: 1 [4480/347990 (1%)]	Loss per example: 0.005438	Loss: 3.480098
Train Epoch: 1 [5120/347990 (1%)]	Loss per example: 0.005381	Loss: 3.443710
Train Epoch: 1 [5760/347990 (2%)]	Loss per example: 0.005403	Loss: 3.458076
Train Epoch: 1 [6400/347990 (2%)]	Loss per example: 0.005172	Loss: 3.310017
Train Epoch: 1 [7040/347990 (2%)]	Loss per example: 0.005620	Loss: 3.596819
Train Epoch: 1 [7680/347990 (2%)]	Loss per example: 0.005306	Loss: 3.396065
Train Epoch: 1 [

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Validation set: Average loss: 0.0938, Accuracy: 4303/4410 (98%)

Train Epoch: 2 [0/347990 (0%)]	Loss per example: 0.000744	Loss: 0.476158
Train Epoch: 2 [640/347990 (0%)]	Loss per example: 0.000449	Loss: 0.287550
Train Epoch: 2 [1280/347990 (0%)]	Loss per example: 0.000405	Loss: 0.259183
Train Epoch: 2 [1920/347990 (1%)]	Loss per example: 0.000461	Loss: 0.294724
Train Epoch: 2 [2560/347990 (1%)]	Loss per example: 0.000818	Loss: 0.523501
Train Epoch: 2 [3200/347990 (1%)]	Loss per example: 0.000382	Loss: 0.244219
Train Epoch: 2 [3840/347990 (1%)]	Loss per example: 0.001101	Loss: 0.704554
Train Epoch: 2 [4480/347990 (1%)]	Loss per example: 0.000482	Loss: 0.308577
Train Epoch: 2 [5120/347990 (1%)]	Loss per example: 0.000566	Loss: 0.362511
Train Epoch: 2 [5760/347990 (2%)]	Loss per example: 0.000355	Loss: 0.227407
Train Epoch: 2 [6400/347990 (2%)]	Loss per example: 0.000643	Loss: 0.411336
Train Epoch: 2 [7040/347990 (2%)]	Loss per example: 0.000751	Loss: 0.480804
Train Epoch: 2 [7680/34799

In [ ]:
def test():
  model.eval()
  correct = 0
  for data, target in test_loader:
      with torch.no_grad():
          data, target = Variable(data), Variable(target)
          if use_gpu:
              data = data.cuda()
              target = target.cuda()
          output = model(data)
          pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
          f1=f1_score(target.long().cpu().numpy(),pred.cpu().numpy(),average="macro")
          correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  print('\nTest set: F1-score: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      f1, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

In [ ]:
test()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3448: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "



Test set: F1-score: 1.0000, Accuracy: 12513/12630 (99%)



In [ ]:
pwd

'/content/data'

In [ ]:
cd ..

/content


In [ ]:
rm -rf data